In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
try:
    import nltk
except:
    !pip install nltk
    import nltk

In [73]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rounaksarkar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [74]:
import tensorflow as tf
import random
import json

In [75]:
with open('intents.json') as json_data:
    data = json.load(json_data)

In [76]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [77]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']

# loop through each sentence in our intents patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w=nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w,intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [78]:
# Perform stemming and lemmatization on the data
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')

words= [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
lemmatizer = WordNetLemmatizer()
words= [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# remove duplicates and sort
words= sorted(list(set(words)))

# sort classes
classes= sorted(list(set(classes)))

# documents= combination between patterns and intents
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique lemmatized words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'medium', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rounaksarkar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [79]:
import numpy as np

# Create training data (input and output pairs)
training = []
output = []

# Create an empty array for output
output_empty = [0] * len(classes)

# Create training set, bag of words for each sentence
for doc in documents:
    # Initialize bag of words
    bag = [0] * len(words)
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # Create bag of words array with 1, if word match found in current pattern
    for w in pattern_words:
        if w in words:
            bag[words.index(w)] = 1
    # Output is 1 for current tag and 0 for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    # Append bag of words and output row to training list
    training.append([np.array(bag, dtype=np.float32), np.array(output_row, dtype=np.float32)])

# Shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# Create training and testing lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [80]:
model= tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=50, verbose=1)
model.save('chatbot_model.h5')

Epoch 1/100
1/1 [==============================] - 0s 159ms/step - loss: 2.0584 - accuracy: 0.2593
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 2.0485 - accuracy: 0.2593
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 2.0387 - accuracy: 0.2593
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 2.0289 - accuracy: 0.2963
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 2.0192 - accuracy: 0.2963
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 2.0096 - accuracy: 0.2963
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0000 - accuracy: 0.2963
Epoch 8/100
1/1 [==============================] - 0s 3ms/step - loss: 1.9905 - accuracy: 0.2963
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 1.9810 - accuracy: 0.2963
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 1.9716 - accuracy: 0.2963
Epoch 11/100
1/1 [=========

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [82]:
import pickle
pickle.dump({'words':words, 'classes':classes},open('chatbot_model.h5','wb'))


In [83]:
from keras.models import load_model
import os

model_path = 'chatbot_model.h5'
if os.path.exists(model_path):
    try:
        model = load_model(model_path)  # Load the trained model
    except OSError as e:
        print("Unable to load model:", e)
else:
    print("Model file not found at path:", model_path)


Unable to load model: Unable to open file (file signature not found)


In [84]:
# Restoring all data structures
import pickle
data = pickle.load(open("chatbot_model.h5", "rb"))
words = data['words']
classes = data['classes']

In [85]:
with open('intents.json') as file:
    data = json.load(file)

In [86]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [87]:
# Returning the bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # Tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # Bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

In [88]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bag_of_words(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [89]:
def response(sentence):
    results.classify(sentence)
    # If we have a classification then find the matching intent tag
    if results:
        # loop as long as there re matches to process
        while results:
            # for each intent key, find the matching tag
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [91]:
response('where are you located?') 

NameError: name 'results' is not defined